# Introduction to the NCAS CF Data Tools, cf-python and cf-plot

## Context and learning objectives

### What are the NCAS CF Data Tools and why do they all have 'cf' in the name?

The _NCAS CF Data Tools_ are a suite of complementary Python libraries which are designed to facilitate working with data for research in the earth sciences and aligned domains. The two that are of most relevance to the average user, and those wanting to process, analyse and visualise atmospheric data, are *cf-python* (https://ncas-cms.github.io/cf-python/) and *cf-plot* (https://ncas-cms.github.io/cf-plot/build/). We will be focusing on use of cf-python and cf-plot today.

The 'cf' in the names of the NCAS CF Data Tools corresponds to the _CF Conventions_, a metadata standard, because they are built around this standard in the form of using the CF Data Model, which as well as performance is considered a 'unique selling point' of the tools.


### What are the CF Conventions?

The _CF Conventions_, usually referred to in this way but also know by the full name of the **C**limate and **F**orecast (CF) metadata conventions, are a metadata standard which is becoming the de-facto convention to cover the description of geoscientific data so that sharing and intercomparison is simpler. See https://cfconventions.org/ for more information.


### What are we going to learn in this session?

Our **learning aim** is to be able to use the NCAS CF Data Tools Python libraries, namely cf-python and cf-plot to process, analyse and visualise netCDF and PP datasets, whilst appreciating the context and 'unique selling point' of the libraries as being built to use the CF Conventions, a metadata standard for earth science data, to make it simpler to do what you want to do with the datasets, by working on top of a Data Model for CF.

We have **six distinct objectives**, matching the sections in this notebook and in the practical notebook you will work through. By the end of this lesson you should be familiar and have practiced using cf-python and cf-plot to:

1. read dataset(s) and view the (meta)data at different detail levels;
2. edit the (meta)data and write out the edited version to file;
3. reduce datasets by subspacing and collapsing;
4. visualise datasets as contour and vector plots;
5. analyse data: applying mathematical and statistical operations and plotting trends;
6. change the underlying grid of data through regridding.

<div class="alert alert-block alert-info">
<i>Note:</i> much of what you can do with cf-python you can do with the xarray library. Use whichever approach, the cf-python/cf-plot way, or the xarray way, works best for you! However, we want to emphasise that the NCAS CF Data Tools are built around the CF Conventions whereas xarray is not, so cf-python and cf-plot offer better metadata awareness to xarray, which could be a core advantage to our approach for users in/from geoscience. (If you have suggestions for how we can improve cf-python and/or cf-plot for you or your work, please let us know through the Issue Trackers linked at the end of this Notebook.)
</div>

***

## Setting up

**In this section we set up this Notebook, import the libraries and check the data we will work with, ready to use the libraries within this notebook.**

Import cf-python and cf-plot:

In [ ]:
import cfplot as cfp
import cf

Run some set up for nice outputs in this Jupyter Notebook (not required in interactive Python or a script):

In [ ]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

Inspect the versions of cf-python and cf-plot and the version of the CF Conventions those are matched to:

In [ ]:
print("cf-python version is:", cf.__version__)
print("cf-plot version is:", cfp.__version__)
print("CF Conventions version is:", cf.CF())

<div class="alert alert-block alert-info">
<i>Note:</i> you can work with data compliant by any other version of the CF Conventions, or without (much) compliance, but the CF Conventions version gives the maximum version that these versions of the tools understand the features of.
</div>

Finally, see what datasets we have to explore:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' preceeeds a shell command, so this is a terminal command and not Python
</div>

In [ ]:
!ls ../data

***

## 1. Reading dataset(s) and viewing the (meta)data at different detail levels

**In this section we look at a basic use of cf-python, reading in one or more datasets from file and inspecting the data and the metadata at different levels of detail to suit the amount of information you want to see.**

<div class="alert alert-block alert-info">
<i>Note:</i> In cf-python and when discussing related code and datasets, we use terminology from the CF Data Model (for more detail see: <a href="url">https://ncas-cms.github.io/cf-python/cf_data_model.html</a>). For example cf-python methods are named in relation to concepts from this data model. We don't have time to cover this in detail but for this session it is useful to know the following terms:

<ul>
    <li><b>field</b>: a self-contained cf-python object corresponding to a netCDF data variable with all of its (CF) metadata attached;</li>
    <li><b>field list</b>: a list of lields (see above), stored as its own cf-python object 'FieldList' which is similar to a Python list;</li>
    <li><b>coordinate</b>: a (CF) metadata concept which corresponds to netCDF coordinate variables. One or more coordinates are defined on every field as either 'dimension' or 'auxiliary' coordinate objects in cf-python.</li>
</ul>
</div>

The examples from this section should help you to familiarise yourself with these terms and their practical usage.

### a) Reading in data and extracting the _field_ of interest

Read a chosen data file. Sometimes datasets have descriptive names but this one doesn't, so let's find out what it is!

In [ ]:
fieldlist = cf.read("../data/data1.nc")

See the 'fieldlist' that cf-python interprets from the data read in:

In [ ]:
fieldlist

Select a particular field from the fieldlist of interest:

In [ ]:
field = fieldlist[0]

### b) Inspecting the _field_ of interest with different amounts of detail

View the field with **minimal detail**, i.e. a one-line summary:

In [ ]:
field

Or you can view it with a **medium level of detail** with the Python built-in `print` function:

In [ ]:
print(field)

A final option is to view it with **maximal detail** using the `dump()` method:

In [ ]:
field.dump()

### c) Inspecting a metadata _construct_ e.g. _coordinate_ from the _field_ of interest

Use the same approach to view a particular metadata aspect, for example the latitude coordinate:

In [ ]:
lat = field.coordinate("latitude")

In [ ]:
lat

In [ ]:
print(lat)

In [ ]:
lat.dump()

### d) Inspecting a data array of interest

Likewise, the same approach works to view the data itself in the field (i.e. the underlying arrays). First, grab the data from the field with the `data` attribute:

In [ ]:
data = field.data

Then view it in a chosen level of detail as with the above objects:

In [ ]:
data

In [ ]:
print(data)

In [ ]:
data.dump()

If you want to see more of the data array itself, you can access it with the `array` attribute. Beware, for real-life datasets:

* this will be large and Python will likely truncate it so your screen isn't spammed with sub-arrays of values!
* it is computationally intensive to access the underlying data array if it is large, especially if it is multi-dimensional, so your computer will often have to work hard to get the array, so use the `array` method sparingly (only when needed)!

In [ ]:
data.array

***

## 2. Editing the (meta)data and writing out the edited version to file

**In this section we demonstrate how to change the data that has been read-in from file, both in terms of the data arrays and the metadata that describes it, and then how to write data back out to file with a chosen name, so that you can see how cf-python can be used to edit data or to make new data.**

Using the same data file from te previous section, let's say we want to change the data and metadata of this. As-is the field and its data are:

In [ ]:
print("Field is:", field, "Data is:", data, sep="\n\n")

### a) Changing the underlying data

To change the data, use assignment to the relavant index or indices. For example, to change all values we can use the special index of an ellipsis like so, in this case changing them all to an identical scalar value:

In [ ]:
data[...] = 10.0
data

In [ ]:
print(data)

Or could change more specifically just one sub-array of these to a different value

In [ ]:
data[0, 0, 0] = 3.0
data.array

Instead of setting the whole sub-array to one value, you can set the whole array to your precise specification, for example:

In [ ]:
data[0, 0, 0] = range(320)
data.array

### b) Changing some metadata

To change metadata, first get the metadata you want to change as an object. One of the most flexible ways to do so is to use the `construct` method and as an argument specify the name of the coordinate you are interested in:

In [ ]:
pressure = field.construct("pressure")

In [ ]:
print(pressure)
print(pressure.data)

You can inspect the units specifically using the `units` attribute:

In [ ]:
print(pressure.units)

Let's change the units to an equivalent but different unit, the `bar` (out by a factor of 1000), as an example:

In [ ]:
pressure.units = "bar"
print(pressure.units)

Notice how the data has been converted to account for the new units - cf-python's metadata awareness makes contextual changes like this so we don't have to do it manually!

In [ ]:
print(pressure.data)

Note how the pressure units are changed in the field too, since we edited the same object in a Pythonic sense:

In [ ]:
print(field)

### c) Writing a (list of) fields out to a file

We changed some metadata (units) and the data itself from our dataset read-in from file. Let's write the new data out
as a new file and read it back in to show that it has been changed relative to the original. You write files out to disk using the `write` function with an argument giving the path, including the name (it can _just_ be the name to write a file to the current working directory), you want to create the file to:

In [ ]:
cf.write(field, "../data/data1-updated.nc")

See that it was written out to the directory we specified:

<div class="alert alert-block alert-info">
<i>Note:</i> in a Jupyter Notebook, '!' preceeeds a shell command, so this is a terminal command and not Python
</div>

In [ ]:
!ls ../data

To check it wrote out the edited version from this Notebook, we can read the file back in and inspect it again:

In [ ]:
updated_fieldlist = cf.read("../data/data1-updated.nc")
reread_field = updated_fieldlist[0]

See what `g` is by medium detail inspection:

In [ ]:
print(reread_field)
print(reread_field.data)

Notice the pressure coordinate units are 'bar' as per our change and the first data array item starts with `0.0` and the final one ends with `10.0` as per our change.

***

## 3. Reducing datasets by subspacing and collapsing

**In this section we show how multi-dimensional data can be tamed using cf-python so that you can get a reduced form that can be analysed or plotted, by reducing the dimensions by selecting a subset of point(s) along the axes or collapsing down according to some statistic such as the mean or an extrema.**

Often datasets represent highly multi-dimensional data, for example 4D or higher. Usually we want to find a either a sub-space, or a statistical representation (such as an average or extrema), of the full data array in less dimensions, such as in 3D or 2D or even in the form of a 1D time series or 0D statistic.

We'll demonstrate this with another dataset and field selected from it. This serves as a reminder on concepts from the first section of the Notebook:

<div class="alert alert-block alert-info">
<i>Note:</i> here we are reading in a file in the 'PP' format, which is a file format originating from the Met Office that can often be encountered in geoscience, like netCDF, hence the file extension '.pp'. You can read and process PP files using cf-python exactly the same way you do for netCDF files, so you do not need to concern yourself with the difference in file format in your code.
</div>

In [ ]:
field2 = cf.read("../data/aaaaoa.pmh8dec.pp")[2]
print(field2)

In this case, see the numbers representing axes sizes on the 'Data' line: we have a 3D field where the axes sizes are 17 for air pressure, 30 for grid latitude and 24 for grid latitude. To reduce this to a 2D form, we need to take one of the non-zero axes and convert it to size 1. This can be done either by **subspacing** or by **statistically collapsing** it.

### a) Subspacing using metadata conditions

Use the `subspace` method to find a subspace of a field, the output of which is another field reduced down in the way specified by the method arguments:

In [ ]:
field2_subspace1 = field2.subspace(air_pressure=1000.0000610351562)  # taking first value
print(field2_subspace1)

Let's do the same but to subspace to reduce the data along a different axis, this time `grid_latitude`:

In [ ]:
field2_subspace2 = field2.subspace(grid_latitude=-5.279999852180481)  # taking the last value
print(field2_subspace2)

### b) Subspacing using indexing, including equivalency to the above

We can also use indexing to do a subspace. So, instead of picking out a given value from the printed information,
we can use a specific Python index to pick it out.

The `air_pressure` coordinate is listed as first in axes order, so you must specify the index at the first position of the three i.e. `[<index>, :, :]` where `:` means to not take a subspace and leave the whole axis as it was. For example, the following takes the first (position 0 in Python indexing) value of the `air_pressure`:

In [ ]:
field2_subspace1_by_index = field2[0, :, :]  # taking first value from first coordinate
print(field2_subspace1_by_index)

To prove that this is the same field as we got using the direct `subspace` method from the sub-section above, namely `h.subspace(air_pressure=1000.0000610351562)`, we can use the `equals` method, which states whether one field is identical to another:

In [ ]:
field2_subspace1_by_index.equals(field2_subspace1)

Similarly, the `grid_latitude` coordinate is listed as second in axes order, so you must specify the index at the second position of the three i.e. `[:, <index>, :]`. Let's take the last (position -1 in Python indexing) value from this, like so:

In [ ]:
field2_subspace2_by_index = field2[:, -1, :]  # taking last value from second coordinate
print(field2_subspace2_by_index)

Again, to prove that this is the same field as we got as using the direct `subspace` approach previously with `h.subspace(grid_latitude=-5.279999852180481)`:

In [ ]:
field2_subspace2_by_index.equals(field2_subspace2)

You can do multiple subspaces at once via either of the methods above, so for example you can combine the two separate subspaces into one call:

In [ ]:
field2_subspace3 = field2.subspace(air_pressure=1000.0000610351562, grid_latitude=-5.279999852180481)
field2_subspace3_by_index = field2[0, -1, :]

This results in (in both cases):

In [ ]:
field2_subspace3.equals(field2_subspace3_by_index)
print(field2_subspace3)

### c) Statistical collapses

Instead of extracting the data at a particular value from the `air_pressure` dimension coordinate, we might want to collapse the data down according to a representative statistic covering all of those values, such as an average or extrema value. We do this with the `collapse` method.

Say we want to get the _mean_ of all of the air pressure values to reduce that coordinate from having the 17 values to just one mean representation, we would do this as follows:

In [ ]:
field2_collapse1 = field2.collapse("air_pressure: mean")  # taking mean of the 17 values for air pressure
print(field2_collapse1)

Another example is taking the _minimum_ of all of the values across the grid latitudes:

In [ ]:
field2_collapse2 = field2.collapse("grid_latitude: minimum")  # taking minimum of the 30 values for the grid latitude
print(field2_collapse2)

Equivalently, you can specify the axes via an `axes` argument when the coordinate to collapse along is one of `X`, `Y`, `Z` or `T`. So this `collapse` call will give the same result:

In [ ]:
field2_collapse3 = field2.collapse("minimum", axes="Y")  # taking minimum of the 30 values for the grid latitude
print(field2_collapse3)

Proving that this gives the same result as with the `"grid_latitude: minimum"` argument:

In [ ]:
field2_collapse3.equals(field2_collapse2)

***

## 4. Visualising datasets as contour and vector plots

**In this section we demonstrate how to plot using cf-plot the data we have read and then processed and/or analysed using cf-python, notably showing how to create contour plots and vector plots as examples of some of the available plot types.**

For this section let's use other fields within the fieldlist we read in for section one, where we investigated the first of the four fields (index 0) in that section:

In [ ]:
fieldlist

This time let's investigate the other (final) three fields. First we unpack these to variables to save us applying the indexing multiple times (note we assign variable names `fieldN` to a number `N` incremented with fields read and assigned to variable names from the start of the Notebook, not with numbers corresponding to the index position):

In [ ]:
temp_field = fieldlist[1]
wind_u_field = fieldlist[2]
wind_v_field = fieldlist[3]

Now let's see what the fields are like, with medium detail, using a new-line separated print to break up the outputs:

In [ ]:
print("'temp_field' is:", temp_field, "\n")
print("'wind_u_field' is:", wind_u_field, "\n")
print("'wind_v_field' is:", wind_v_field, "\n")

### a) Making a contour plot

All of these fields, as we can see from the latitude and longitude value ranges, span the whole globe in extent. Another quick way to gauge this, and plenty of other information about a field notably the data itself and its form and pattern(s), is to plot it.

To make a contour (level) plot of a field, use the `con` function of cf-plot. It requires as its one positional argument a field with only two axes which are greater than size one in order to have an effective 2D field, often a 'slice' i.e. subspace or collapse of a higher-dimensional field, it can plot.

For example, for our fields above we need to reduce one axis down to achieve this, so for example we can reduce the `pressure` coordinate. Let's take a subspace at a given pressure (we take 1000 mbar, the highest in the field), and plot it:

In [ ]:
cfp.con(temp_field.subspace(pressure=1000.0))

Let's see what the equivalent plot would be for the subspace over a different pressure, this time the lowest 1 mbar:

In [ ]:
cfp.con(temp_field.subspace(pressure=1.0))

### b) Customising the (contour) plot

cf-plot provides numerous ways to customise a specific plot such as those above. We demonstrate just a small number here.

The second plot from the previous sub-section, in particular, has contour lines which are so dense they obscure the world map and data pattern so let's hide those which you do by specifying `lines=False` to the `con` call:

In [ ]:
cfp.con(temp_field.subspace(pressure=1.0), lines=False)

Let's also do some further customisation of the plot above, as follows:

- use a different map projection;
- set a different colour map for the data;
- move the colour bar to be vertical not horizontal;
- add a title:

In [ ]:
cfp.mapset(proj="robin") # changes the projection
cfp.cscale("plasma")  # changes the colour scale/map
cfp.con(
    temp_field.subspace(pressure=1.0),
    lines=False,
    colorbar_orientation="vertical",  # tells cf-plot to make a vertical colour bar (horizontal is the default)
    title="Air temperaure field"  # adds the specified title
)

We might want to view a smaller section of the data, say over Australia only. We can do this in two ways:

1. by reducing the data further; or
2. by specifying to cf-plot to only show a smaller window on the data.

For (1), instead of a subspace on just pressure, we also suspace on X and Y i.e. longitude and latitude, to reduce those axes. We can use `cf.wi` which specifies a 'within' condition. First, define the extrema of both of these we want to view, for example these are those longitudes and latitudes which provide a view centered around Australia:

In [ ]:
cfp.mapset() # resets the projection
lonmin = 110
lonmax = 155
latmin = -45
latmax = -5

Now do method (1) from above, via a subspace on three axes, pressure (as before) as well as longitude and latitude (new). Note we could do each subspace via a separate call, but they can also be done together in one call like so:

In [ ]:
# Do further subspacing to reduce the latitude and longitude with cf-python
fsub_all = temp_field.subspace(
    pressure=1.0,
    longitude=cf.wi(lonmin, lonmax),
    latitude=cf.wi(latmin, latmax)
)

# Plot the above
cfp.con(fsub_all, lines=False)

Method (2) can be achieved by setting the keyword arguments to the cf-plot `mapset` method like so:

In [ ]:
cfp.mapset(lonmin=lonmin, lonmax=lonmax, latmin=latmin, latmax=latmax)
cfp.con(temp_field.subspace(pressure=1.0), lines=False)

Note the differences in colour shades between the two plots from the different approaches are because in approach (2) cf-plot normalises the colour map extent of the data to only account for the area shown.

### c) Making a vector plot with basic customisation

Finally let's make some vector plots. Notice our final two fields from the read-in fieldlist are in eastward and northward components, therefore can be combined into vectors from these vector components:

In [ ]:
print(wind_u_field)
print(wind_v_field)

Again we need to subspace in pressure so that we have an effective 2D not 4D space that can be visualised as a contour plot.
This time let's take our subspace at 500 mbar pressure:

In [ ]:
u = wind_u_field.subspace(pressure=500.0)
v = wind_v_field.subspace(pressure=500.0)

To make a vector plot using cf-plot, use the `vect` function:

In [ ]:
cfp.mapset()  # reset the mapping settings, including map projection
cfp.vect(u=u, v=v)

We can mostly just see black here! This is expected and because we need to customise the vector spacing. There are clearly too many vectors shown they overwhelm the plot area, so use the `scale` and `stride` keyword arguments to `vect` to edit the size and spacing of the vector arrows respectively:

In [ ]:
cfp.vect(u=u, v=v, scale=100, stride=4)  # much better - found these values after playing around with values

Like for contour plotting, we can focus in on specific areas of the data in space via the two methods covered there. In the vector plot case, note we may need to adjust the vector spacing and size again for optimal viewing.

In [ ]:
cfp.mapset(lonmin=lonmin, lonmax=lonmax, latmin=latmin, latmax=latmax) # also resets the projection to the default 'cyl'
cfp.vect(u=u, v=v)

Trial and error for vector size and spacing gives a nicer result for the above plot with:

In [ ]:
cfp.vect(u=u, v=v, scale=50, stride=1)  # better than the above!

***

## 5. Analysing data: applying mathematical and statistical operations and plotting trends

**In this section we demonstrate how to do some data analysis including performing arithmetic and statistical calculations on the data, showing how cf-python's CF Conventions metadata awareness means that the metadata is automatically updated to account for the operations that are performed.**

As well as the statistics you can calculate and explore from collapsing fields in section (3c), you can ...

### a) Applying mathematics e.g. arithmetic and trigonometry on fields

We will use another dataset to demonstrate this, to remind of the `read` function and fieldlist unpacking from section one:

In [ ]:
monthly_field = cf.read("../data/IPSL-CM5A-LR_r1i1p1_tas_n96_rcp45_mnth.nc")[0]
print(monthly_field)
print(monthly_field.data)

You can perform arithmetical operations on fields using the usual operators, e.g. let's multiply the field's underlying data by 2 and subtract 10 from it to illustrate:

In [ ]:
double_minus_ten_monthly_field = 2 * monthly_field - 10
print(double_minus_ten_monthly_field)
print(double_minus_ten_monthly_field.data)

You can apply mathematical operations too via various available methods. Let's try some rounding and trigonometry as an illustration:

In [ ]:
round_monthly_field = monthly_field.round()
print(round_monthly_field.data)

In [ ]:
cosine_monthly_field = monthly_field.cos()
print(cosine_monthly_field)

Note the units change appropriately with some operations, again showcasing cf-python's metadata awareness.

### b) Line plotting

Often we want to pick out statistical trends from a subspace of the data. cf-python has season-selecting and season-collapsing methods to help you determine information on a per-season basis (as well as month-selecting methods to do the same for specific months, though we don't try any of those here).

Firstly, let's show a line plot of our data averaged across the whole spatial area, so we can see the overall pattern. You can produce a line plot with cf-plot using the `lineplot` function, where the argument should be a field with a 1D series (ignoring any 1D axes such as latitude and longitude in our case which after the collapse become 1D):

In [ ]:
spatial_mean_monthly_field = monthly_field.collapse("area: mean")
print(spatial_mean_monthly_field)

cfp.lineplot(spatial_mean_monthly_field)

### c) Calculating seasonal trends

This data looks like it has trends on different time scales e.g. due to seasons. Let's pick some of those out using cf-python. We select two of the four seasons, using the `djf` and `jja` methods like so:

In [ ]:
get_djf_season = cf.djf()  # specific collapse type for the months of December, January, February
get_jja_season = cf.jja()  # specific collapse type for the months of June, July, August

Then we collapse on these only using the `group` keyword argument to the `collapse` method, which we call a 'grouped collapse':

In [ ]:
djf_season_mean = spatial_mean_monthly_field.collapse("T: mean", group=get_djf_season)  # mean across DJF season
cfp.lineplot(djf_season_mean)

In [ ]:
jja_season_mean = spatial_mean_monthly_field.collapse("T: mean", group=get_jja_season)  # mean across JJA season
cfp.lineplot(jja_season_mean)

### d) Plotting the seasonal trends on one (line)plot

To put those seasonal averages into context from the original data, it would be nice to plot them on top of the original line plot. We can do that using cf-plot, as an example of some more advanced cf-plot plotting capability. If you want to plot multiple aspects on one plot in this way, wrap the calls to the functions to plot such as `con` or `vect` or `lineplot` with the opening and closing functions `gopen` and `gclose`, like so:

In [ ]:
cfp.gopen()
# By adding the 'label' argument, we allow labels for the corresponding line on the plot legend
cfp.lineplot(spatial_mean_monthly_field, label="Original monthly data (spatial mean)")
cfp.lineplot(djf_season_mean, label="Mean over the DJF months of the original spatial mean")
cfp.lineplot(jja_season_mean, label="Mean over the JJA months of the original spatial mean")
cfp.gclose()

***

## 6. Changing the underlying grid of data through regridding

**In this section we demonstrate how to change the underlying grid of the data to another grid which could be a higher- or lower- resolution one, or a completely different grid, which is called regridding or interpolation, and indicate various options cf-python supports for doing this.**

### a) Getting a _source_ field ready to regrid

We read in a precipitation field and inspect it:

In [ ]:
higher_res_field = cf.read("../data/precip_2010.nc")[0]
print(higher_res_field)

To get a feel for the data we have, let's view a basic subspace as a contour plot. Note we use the `blockfill` argument set to `True` which changes the contour plot to plot the data on a cellular basis rather than filled as smoothed contours (whether or not the contour lines are set to be shown from the `lines` argument, in this case we turn those off too):

In [ ]:
cfp.mapset()  # reset
cfp.cscale("precip_11lev")  # using a colour scale good for precipitation data
cfp.con(higher_res_field[0], blockfill=True, lines=False)

### b) Getting the _destination_ field: another field in order to regrid the previous _onto its grid_

Now we read in another precipitation field and inspect it. The key thing to note is that it is lower resolution than the previous one we read:

In [ ]:
lower_res_field = cf.read("../data/model_precip_DJF_means_low_res.nc")[0]
print(lower_res_field)

Again, get a feel for the data we have from a view of a basic subspace, to confirm that it is lower resolution (notice the larger cell blocks than before):

In [ ]:
cfp.cscale("precip_11lev")
cfp.con(lower_res_field[0], blockfill=True, lines=False)

### c) Performing the regrid operation from the source to the destination fields

Now the key step after the previous setup: regridding the first (*source*) field to the grid of the second (*destination*) field. We use the `regrids` method of cf-python to do this, where the `s` in the name stands for spherical for spherical regridding, as opposed to Cartesian regridding, also possible with cf-python.

We are going to do two calls, demonstrating two different methods of interpolation, namely the `patch` and `conservative` methods. Assign these to variables so we can compare them next:

In [ ]:
patch_regridded_field = higher_res_field.regrids(lower_res_field, method="patch")
conservative_regridded_field = higher_res_field.regrids(lower_res_field, method="conservative")

These two methods do not give the same results!

In [ ]:
patch_regridded_field.equals(conservative_regridded_field)

Recall how to do field arithmetic. We use subtraction to indicate the difference between the two regridded outcomes which differ only by the interpolation method used to regrid:

In [ ]:
difference_field = patch_regridded_field - conservative_regridded_field

### d) Finally, some more advanced cf-plot plotting to compare the source, destination, and regridded results

As with the previous section, let's end by doing more advanced cf-plot plotting to show all of the relevant fields to help us to understand what the regridding did.

Again we are using `gopen` and `gclose` wrapped around our calls to plot. This time we use `gpos` function calls which tell cf-plot exactly where we want to place each plot on the overall canvas. In this case, from our `x` and `y` extrema inputs `xmin`, `xmax`, `ymin` and `ymax`, we are building an effective 4 x 4 grid to place the four contour plots we will show.

There is a lot to take in here, but the main thing is to understand that we use `con` to generate four contour plots corresponding to different stages of our regridding, telling cf-plot to place them at one of the four places in a 4 x 4 grid, with some customisation of colour scales and colour bar levels.

Note here we are taking the [0] index subspace of all of the fields to plot, but we can use any other subspace of the data to view instead:

In [ ]:
cfp.cscale("viridis")  # reset cmap to perceptually uniform scale
cfp.gopen(rows=2, columns=2, figsize=[7, 15])  #, bottom=0.1, top=0.85)

# Configure first three plots
cfp.mapset()  # reset from previous plots in the notebook - standalone don't need this
cfp.levs(min=0, max=500, step=50)

# First three plots
cfp.gpos(xmin=0.1, xmax=0.5, ymin=0.55, ymax=1.0)
cfp.cscale("precip_11lev")
cfp.con(
    higher_res_field[0], blockfill=True, lines=False,
    title="Precipitation field\nbefore regridding",
)
cfp.gpos(xmin=0.55, xmax=0.95, ymin=0.55, ymax=1.0)
cfp.con(
    patch_regridded_field[0], blockfill=True, lines=False,
    title="...and after regridding with\nthe patch recovery method",
)
cfp.gpos(xmin=0.1, xmax=0.5, ymin=0.1, ymax=0.55)
cfp.con(
    conservative_regridded_field[0], blockfill=True, lines=False,
    title="...and after regridding with\nthe conservative method",
)
cfp.gpos(xmin=0.55, xmax=0.95, ymin=0.1, ymax=0.55)

# This final plot is showing the diff so is a bit different, apply some cf-plot plotting configuration
cfp.levs()
cfp.cscale("BlueDarkRed18")  # is a diff so use a diverging colour map
cfp.con(
    difference_field[0], blockfill=True, lines=False,
    title="Difference between the two\nregridding methods is:",
)
cfp.gclose()

More generally, using regridding with cf-python you can convert from one grid to another very different grid, e.g. from a tripolar to a regular or rotated latitude-longitude grid or vice-versa, including to/from unstructured grids (UGRID grids).

***

## Conclusion and recap of learning objectives

The NCAS CF Data Tools are a suite of Python libraries which are designed to facilitate working with data for research in the earth sciences and aligned domains. We learnt today about the cf-python (https://ncas-cms.github.io/cf-python/) and cf-plot (https://ncas-cms.github.io/cf-plot/build/). The 'cf' in the names of the NCAS CF Data Tools corresponds to the CF Conventions, a metadata standard becoming the de-facto convention across geoscience to cover the description of data so that sharing and intercomparison is simpler.

Our **learning aim** was to be able to use the NCAS CF Data Tools Python libraries, namely cf-python and cf-plot to process, analyse and visualise netCDF and PP datasets, whilst appreciating the context and 'unique selling point' of the libraries as being built to use the CF Conventions, a metadata standard for earth science data, to make it simpler to do what you want to do with the datasets, by working on top of a Data Model for CF.

For our **learning objectives**, we practiced using cf-python and cf-plot to:

* read dataset(s) and view the (meta)data at different detail levels;
* edit the (meta)data and write out the edited version to file;
* reduce datasets by subspacing and collapsing;
* visualise datasets as contour and vector plots;
* analyse data: applying mathematical and statistical operations and plotting trends;
* change the underlying grid of data through regridding.

***

## Where to find more information and resources on the NCAS CF Data Tools

Here are some links relating to the NCAS CF Data Tools and this training.

* This training, with further material, is hosted online and there are instructions for setting up the environment so you can work through it in your own time: https://github.com/NCAS-CMS/cf-tools-training.
* The cf-python documentation lives at https://ncas-cms.github.io/cf-python/.
* The cf-python code lives on GitHub at https://github.com/NCAS-CMS/cf-python. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-python/issues.
* The cf-plot documentation lives at https://ncas-cms.github.io/cf-plot/build/.
* The cf-plot code lives on GitHub at https://github.com/NCAS-CMS/cf-plot. There is an Issue Tracker to report queries or questions at https://github.com/NCAS-CMS/cf-plot/issues.
* There is a technical presentation about the NCAS CF Data Tools avaialble from https://hps.vi4io.org/_media/events/2020/summer-school-cfnetcdf.pdf.
* The website of the CF Conventions can be found at https://cfconventions.org/.
* The landing page for training into the CF Conventions is found here within the website above: https://cfconventions.org/Training/.

If you have any queries after this course, please either use the Issue Trackers linked above or you can email me at: sadie.bartholomew@ncas.ac.uk.

***